In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import warnings

warnings.filterwarnings("ignore")

# Définition de fonctions

In [14]:
def load_data(file_path, separator, columns_to_drop=[]):
    df = pd.read_csv(file_path, sep=separator)
    if columns_to_drop:
        df = df.drop(columns=columns_to_drop)
    return df

def zscore(data_frame, column_name, threshold=2):
    z_scores = (data_frame[column_name] - data_frame[column_name].mean()) / data_frame[column_name].std()
    is_outlier = (z_scores > threshold) | (z_scores < -threshold)
    outliers = data_frame[is_outlier]
    return outliers

# Fichier DisponibilitéAlimentaire_2017

In [15]:
df_dispalim = pd.read_csv(
    r"C:\Users\melvin.derouk\Desktop\P9\data\DisponibiliteAlimentaire_2017.csv",
    sep=","
    )

In [16]:
df_dispalim.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [17]:
df_dispalim.columns = df_dispalim.columns.str.strip()

In [18]:
#renommage des colonnes Zone & Code zone

df_dispalim = df_dispalim.rename(columns={'Zone': 'Pays', 'Code zone': 'Code Pays'})

In [19]:
#suppression colonnes inutiles 

col_supp_dispalim = ['Code Élément', 'Année', 'Code Domaine', 'Domaine', 'Symbole', 'Code année', 'Description du Symbole']

df_dispalim = df_dispalim.drop(columns=col_supp_dispalim)

In [20]:
#filtre du df sur les produits de type volaille

filt_produit = df_dispalim['Produit'].str.contains('poulet|volaille', case=False, regex=True)

df_poulet = df_dispalim[filt_produit]

df_poulet.head(5)

,Code Pays,Pays,Élément,Code Produit,Produit,Unité,Valeur
651,2,Afghanistan,Production,2734,Viande de Volailles,Milliers de tonnes,28.0
652,2,Afghanistan,Importations - Quantité,2734,Viande de Volailles,Milliers de tonnes,29.0
653,2,Afghanistan,Variation de stock,2734,Viande de Volailles,Milliers de tonnes,0.0
654,2,Afghanistan,Disponibilité intérieure,2734,Viande de Volailles,Milliers de tonnes,57.0
655,2,Afghanistan,Pertes,2734,Viande de Volailles,Milliers de tonnes,2.0


In [21]:
#multiplication de la colonne valeurs pour obtenir des tonnes

df_poulet['Élément'].unique()

multiplication_col_dispAlim = [
    "Aliments pour animaux",
    "Autres Utilisations",
    "Disponibilité intérieure",
    "Exportations - Quantité",
    "Importations - Quantité",
    "Nourriture",
    "Pertes",
    "Production",
    "Semences",
    "Traitement",
    "Variation de stock"
]

df_poulet.loc[df_poulet['Élément'].isin(multiplication_col_dispAlim), 'Valeur'] *= 1000

df_poulet = df_poulet.drop(columns=["Unité", "Code Produit"])

In [22]:
#renommage des valeurs de la colonne "Element" pour ajouter l'info de l'ordre de grandeur
 
correspondance_elements = {
    "Aliments pour animaux": "Aliments pour animaux(Tonnes)",
    "Autres Utilisations": "Autres Utilisations(Tonnes)",
    "Disponibilité intérieure": "Disponibilité intérieure(Tonnes)",
    "Exportations - Quantité": "Exportations - Quantité(Tonnes)",
    "Importations - Quantité": "Importations - Quantité(Tonnes)",
    "Nourriture": "Nourriture(Tonnes)",
    "Pertes": "Pertes(Tonnes)",
    "Production": "Production(Tonnes)",
    "Semences": "Semences(Tonnes)",
    "Traitement": "Traitement(Tonnes)",
    "Variation de stock": "Variation de stock(Tonnes)"
}

df_poulet['Élément'] = df_poulet['Élément'].replace(correspondance_elements)

In [23]:
#tri sur les valeurs d'"Element" qui sont pertinentes

valeurs_supp_element = ['Pertes(Tonnes)', 'Résidus', 'Variation de stock(Tonnes)', 'Aliments pour animaux(Tonnes)', 'Semences(Tonnes)', 
                        'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 
                        'Traitement(Tonnes)', 'Autres utilisations (non alimentaire)', 'Alimentation pour touristes'
                        ]

df_poulet = df_poulet[~df_poulet['Élément'].isin(valeurs_supp_element)]

In [24]:
df_poulet.head(6)

,Code Pays,Pays,Élément,Produit,Valeur
651,2,Afghanistan,Production(Tonnes),Viande de Volailles,28000.00
652,2,Afghanistan,Importations - Quantité(Tonnes),Viande de Volailles,29000.00
654,2,Afghanistan,Disponibilité intérieure(Tonnes),Viande de Volailles,57000.00
657,2,Afghanistan,Nourriture(Tonnes),Viande de Volailles,55000.00
658,2,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,Viande de Volailles,1.53
659,2,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Viande de Volailles,5.00


In [25]:
#pivot de la table

df_poulet_pivot = df_poulet.pivot(index='Pays', columns='Élément', values='Valeur')

In [26]:
df_poulet_pivot.head(6)

Élément,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure(Tonnes),Exportations - Quantité(Tonnes),Importations - Quantité(Tonnes),Nourriture(Tonnes),Production(Tonnes)
Pays,,,,,,,
Afghanistan,5.0,1.53,57000.0,NaN,29000.0,55000.0,28000.0
Afrique du Sud,143.0,35.69,2118000.0,63000.0,514000.0,2035000.0,1667000.0
Albanie,85.0,16.36,47000.0,0.0,38000.0,47000.0,13000.0
Algérie,22.0,6.38,277000.0,0.0,2000.0,264000.0,275000.0
Allemagne,71.0,19.47,1739000.0,646000.0,842000.0,1609000.0,1514000.0
Angola,35.0,10.56,319000.0,0.0,277000.0,315000.0,42000.0


In [28]:
df_poulet.info()
df_poulet.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157 entries, 651 to 176398
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Code Pays  1157 non-null   int64  
 1   Pays       1157 non-null   object 
 2   Élément    1157 non-null   object 
 3   Produit    1157 non-null   object 
 4   Valeur     1157 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 54.2+ KB


,Code Pays,Valeur
count,1157.000000,1.157000e+03
mean,126.050130,3.314627e+05
std,72.917318,1.552112e+06
min,1.000000,0.000000e+00
25%,66.000000,3.921000e+01
50%,121.000000,1.000000e+04
75%,188.000000,1.100000e+05
max,276.000000,2.191400e+07


In [29]:
df_poulet.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\CSV\dispo alim", index=False)

# Fichier population_2000_2018

In [30]:
df_population_path = r"C:\Users\melvin.derouk\Desktop\P9\data\Population_2000_2018.csv"
df_population = load_data(df_population_path, ",")

In [31]:
df_population.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [32]:
#suppression colonnes inutiles et renommage

df_population = df_population.drop(columns=['Code année', 'Code Domaine', 'Domaine', "Unité", "Symbole", "Description du Symbole"])

df_population = df_population.rename(columns={'Zone': 'Pays', 'Code zone': 'Code Pays'})

In [33]:
#multiplication de la colonne "Valeur" par 1000 pour obtenir la population totale du pays

df_population['Valeur'] = df_population['Valeur'] * 1000

In [34]:
#filtre du df sur l'année 2017

df_population = df_population[df_population['Année'] == 2017]

In [35]:
df_population.info()
df_population.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 17 to 4409
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Code Pays     236 non-null    int64  
 1   Pays          236 non-null    object 
 2   Code Élément  236 non-null    int64  
 3   Élément       236 non-null    object 
 4   Code Produit  236 non-null    int64  
 5   Produit       236 non-null    object 
 6   Année         236 non-null    int64  
 7   Valeur        236 non-null    float64
 8   Note          1 non-null      object 
dtypes: float64(1), int64(4), object(4)
memory usage: 18.4+ KB


,Code Pays,Pays,Code Élément,Élément,Code Produit,Produit,Année,Valeur,Note
count,236.000000,236,236.0,236,236.0,236,236.0,2.360000e+02,1
unique,NaN,236,NaN,1,NaN,1,NaN,NaN,1
top,NaN,Afghanistan,NaN,Population totale,NaN,Population-Estimations,NaN,NaN,"UNDESA, Population Division – World Population..."
freq,NaN,1,NaN,236,NaN,236,NaN,NaN,1
mean,134.894068,NaN,511.0,NaN,3010.0,NaN,2017.0,3.198362e+07,NaN
std,78.046150,NaN,0.0,NaN,0.0,NaN,0.0,1.318949e+08,NaN
min,1.000000,NaN,511.0,NaN,3010.0,NaN,2017.0,7.930000e+02,NaN
25%,68.750000,NaN,511.0,NaN,3010.0,NaN,2017.0,3.803032e+05,NaN
50%,133.500000,NaN,511.0,NaN,3010.0,NaN,2017.0,5.203510e+06,NaN
75%,198.250000,NaN,511.0,NaN,3010.0,NaN,2017.0,1.930842e+07,NaN


In [36]:
df_population.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\CSV\population", index=False)

# Fichier pib_par_habitants

In [37]:
df_pib_path = r"C:\Users\melvin.derouk\Desktop\P9\data\pib-par-habitant.csv"
df_pib = load_data(df_pib_path, ",")

In [38]:
df_pib.head(5)

,Country Code,Année,Valeur
0,ABW,2017,29007.693003
1,AFG,2017,519.884773
2,AGO,2017,4095.812942
3,ALB,2017,4531.020806
4,AND,2017,38962.880354


In [39]:
#suppression colonnes inutiles et renommage

drop_col_pib = ['Année']
df_pib = df_pib.drop(columns=drop_col_pib)

df_pib.rename(columns={'Country Code': 'iso3', 'Valeur':'PIB (US$)/hab'}, inplace=True)

In [40]:
df_pib.info()
df_pib.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   iso3           251 non-null    object 
 1   PIB (US$)/hab  251 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.1+ KB


,iso3,PIB (US$)/hab
count,251,251.000000
unique,251,NaN
top,ABW,NaN
freq,1,NaN
mean,NaN,16433.538026
std,NaN,24454.380641
min,NaN,292.997631
25%,NaN,2175.189153
50%,NaN,6376.706716
75%,NaN,19308.138162


In [41]:
df_pib.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\CSV\pib", index=False)

# Fichier iso_pays

In [42]:
df_iso = pd.read_csv(
    r"C:\Users\melvin.derouk\Desktop\P9\data\iso_pays.csv", 
    sep=";",
    encoding="ISO-8859-1"
    )

In [43]:
df_iso.head(5)

,iso2,iso3,cnum,country,pays,area,dis_int,landlocked,continent,city_en,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,AW,ABW,533,Aruba,Aruba,193,5.225315,0,America,Oranjestad,...,.,.,.,NLD,.,.,.,.,.,.
1,AF,AFG,4,Afghanistan,Afghanistan,652225,303.761400,1,Asia,Kabul,...,Uzbek,.,.,.,.,.,.,GBR,.,.
2,AO,AGO,24,Angola,Angola,1246700,419.966600,0,Africa,Luanda,...,.,.,.,PRT,.,.,.,.,.,.
3,AI,AIA,660,Anguilla,Anguilla,102,3.798690,0,America,The Valley,...,.,.,.,GBR,.,.,.,.,.,.
4,AL,ALB,8,Albania,Albanie,28748,63.773110,0,Europe,Tirana,...,.,.,.,TUR,.,.,.,.,.,.


In [44]:
#suppression colonnes inutiles

drop_col_iso = ['iso2', 'country', 'cnum', 'langoff_1', 'langoff_2', 'langoff_3', 'lang20_1', 'lang20_2', 'lang20_3', 'lang20_4', 'lang9_1', 'lang9_2', 'lang9_3', 'lang9_4', 'colonizer1', 'colonizer2', 'colonizer3', 'colonizer4', 'city_en', 'landlocked', 'short_colonizer1', 'short_colonizer2', 'short_colonizer3', 'cap', 'maincity', 'citynum']
df_iso = df_iso.drop(columns=drop_col_iso)

In [45]:
#recherche de doublons depuis la colonne pays

doublon_count = df_iso.duplicated(subset=['pays']).sum()
print("Nombre de doublons dans la colonne 'pays':", doublon_count)

mask = df_iso.duplicated(subset=['pays'], keep=False)
doublon_rows = df_iso[mask]
doublon_rows.head(6)

Nombre de doublons dans la colonne 'pays': 13


,iso3,pays,area,dis_int,continent,city_fr,lat,lon
12,AUS,Australie,7686848,1042.8170,Pacific,Sydney,-33.88334,151.166700
13,AUS,Australie,7686848,1042.8170,Pacific,Canberra,-35.30000,149.133000
18,BEN,Bénin,112622,126.2250,Africa,Cotonou,6.40000,2.517000
19,BEN,Bénin,112622,126.2250,Africa,Porto-Novo,6.50000,2.783333
29,BOL,Bolivie,1098581,394.2302,America,La Paz,-16.50000,-68.166660
30,BOL,Bolivie,1098581,394.2302,America,Sucre,-19.08300,-65.250000


In [46]:
#Ne gardez que la derniere occurence des doublons car c'est toujours la capitale en derniere ligne de doublons dans la colonne city

df_iso = df_iso.drop_duplicates(subset=['pays'], keep='last')

doublon_count = df_iso.duplicated(subset=['pays']).sum()
print("Nombre de doublons dans la colonne 'pays':", doublon_count)

Nombre de doublons dans la colonne 'pays': 0


In [47]:
df_iso.info()
df_iso.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225 entries, 0 to 237
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   iso3       225 non-null    object 
 1   pays       225 non-null    object 
 2   area       225 non-null    int64  
 3   dis_int    225 non-null    float64
 4   continent  225 non-null    object 
 5   city_fr    225 non-null    object 
 6   lat        225 non-null    float64
 7   lon        225 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 15.8+ KB


,area,dis_int,lat,lon
count,2.250000e+02,225.000000,225.000000,225.000000
mean,6.055799e+05,182.138241,16.270227,15.573613
std,1.795068e+06,229.634605,25.445527,74.890471
min,7.000000e+00,0.995137,-66.666660,-176.200000
25%,6.256000e+03,29.749670,0.383333,-51.733330
50%,9.100000e+04,113.463100,15.333330,17.166670
75%,4.474000e+05,251.583300,37.500000,51.433330
max,1.707540e+07,1554.244000,64.183330,179.200000


In [48]:
df_iso.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\CSV\iso_pays", index=False)

# Fichier distances_pays 

In [49]:
df_dist_path = r"C:\Users\melvin.derouk\Desktop\P9\data\distances_pays.csv"
df_dist = load_data(df_dist_path, ";")

In [50]:
df_dist.head(5)

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
0,ABW,ABW,0,0,0,0,0,0,0,0,5.225315,5.225315,25.09354,23.04723
1,ABW,AFG,0,0,0,0,0,0,0,0,13257.810000,13257.810000,13168.22,13166.37
2,ABW,AGO,0,0,0,0,0,0,0,0,9516.913000,9516.913000,9587.316,9584.193
3,ABW,AIA,0,0,1,0,0,0,0,0,983.268200,983.268200,976.8974,976.8916
4,ABW,ALB,0,0,0,0,0,0,0,0,9091.742000,9091.742000,9091.576,9091.466


In [51]:
#suppression colonnes inutiles et renommage

drop_col_dist = ['comlang_off', 'comlang_ethno', 'colony', 'comcol', 'curcol', 'col45', 'smctry', 'distcap', 'distw', 'distwces', 'contig']
df_dist = df_dist.drop(columns=drop_col_dist)

df_dist.rename(columns={'iso_d': 'iso3'}, inplace=True)

In [52]:
#filtre du df sur le code iso FRA

df_dist = df_dist[df_dist['iso_o'] == 'FRA']

In [53]:
#recherche de doublons

colonnes_a_verifier = "iso3"
mask_doublons = df_dist.duplicated(subset=colonnes_a_verifier, keep=False)
doublons_dist_iso = df_dist.loc[mask_doublons]

doublons_dist_iso

,iso_o,iso3,dist


In [54]:
#recherche d'outlier avec le z-score

outliers_dist = zscore(df_dist, "dist", threshold=2)

outliers_dist.sort_values('dist', ascending=False)

,iso_o,iso3,dist
15159,FRA,NZL,19263.88
15151,FRA,NFK,17511.67
15019,FRA,AUS,16975.46
15207,FRA,TON,16941.44
15149,FRA,NCL,16757.74
15154,FRA,NIU,16639.50
15073,FRA,FJI,16595.08
15049,FRA,COK,16568.87
15223,FRA,VUT,16346.35
15224,FRA,WLF,16073.01


In [55]:
df_dist.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\CSV\distance", index=False)

# Merges des fichiers

Fichiers dispo alim et population

In [56]:
df_poulet = pd.read_csv(
    r"C:\Users\melvin.derouk\Desktop\P9\CSV\dispo alim",
    sep=","
    )

df_population = pd.read_csv(
    r"C:\Users\melvin.derouk\Desktop\P9\CSV\population",
    sep=","
    )

In [57]:
#merge interne des fichiers avec Pays comme PK

dispalim_pop = pd.merge(df_poulet_pivot, df_population, how='inner', on='Pays', indicator=True)

keys_count = dispalim_pop['_merge'].value_counts()
print(keys_count)

both          172
left_only       0
right_only      0
Name: _merge, dtype: int64


In [58]:
#suppression colonnes inutiles et renommage 

col_supp_dispalim_pop = ['_merge', 'Code Élément', 'Produit', 'Année', 'Code Élément', 'Code Produit', 'Code Pays', 'Élément', 'Note']
dispalim_pop.drop(col_supp_dispalim_pop, axis=1, inplace=True)

dispalim_pop.rename(columns={"Valeur":"Population"}, inplace=True)

In [59]:
dispalim_pop.head(5)

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure(Tonnes),Exportations - Quantité(Tonnes),Importations - Quantité(Tonnes),Nourriture(Tonnes),Production(Tonnes),Population
0,Afghanistan,5.0,1.53,57000.0,NaN,29000.0,55000.0,28000.0,36296113.0
1,Afrique du Sud,143.0,35.69,2118000.0,63000.0,514000.0,2035000.0,1667000.0,57009756.0
2,Albanie,85.0,16.36,47000.0,0.0,38000.0,47000.0,13000.0,2884169.0
3,Algérie,22.0,6.38,277000.0,0.0,2000.0,264000.0,275000.0,41389189.0
4,Allemagne,71.0,19.47,1739000.0,646000.0,842000.0,1609000.0,1514000.0,82658409.0


Fichiers iso_pays et pib

In [60]:
df_pib = pd.read_csv(
    r"C:\Users\melvin.derouk\Desktop\P9\CSV\pib",
    sep=","
    )

df_iso = pd.read_csv(
    r"C:\Users\melvin.derouk\Desktop\P9\CSV\iso_pays",
    sep=","
    )

In [61]:
#merge interne des fichiers avec iso3 comme PK

iso_pib = pd.merge(df_iso, df_pib, how='inner', on='iso3', indicator=True)

keys_count = iso_pib['_merge'].value_counts()
print(keys_count)

both          190
left_only       0
right_only      0
Name: _merge, dtype: int64


In [62]:
#suppression colonne de verif jointures

iso_pib.drop('_merge', axis=1, inplace=True)

In [63]:
iso_pib.head(5)

,iso3,pays,area,dis_int,continent,city_fr,lat,lon,PIB (US$)/hab
0,ABW,Aruba,193,5.225315,America,Oranjestad,12.550000,-70.10000,29007.693003
1,AFG,Afghanistan,652225,303.761400,Asia,Kaboul,34.516670,69.20000,519.884773
2,AGO,Angola,1246700,419.966600,Africa,Luanda,-8.833333,13.25000,4095.812942
3,ALB,Albanie,28748,63.773110,Europe,Tirana,41.333330,19.81667,4531.020806
4,AND,Andorre,453,8.005398,Europe,Andorre-la-Vieille,42.500000,1.50000,38962.880354


Fichier iso_pib et distances

In [64]:
#merge interne des fichiers avec iso3 comme PK

isopib_dist =  pd.merge(iso_pib, df_dist, how='inner', on='iso3', indicator=True)

keys_count = isopib_dist['_merge'].value_counts()
print(keys_count)

both          190
left_only       0
right_only      0
Name: _merge, dtype: int64


In [65]:
#suppression colonne de verif jointures

isopib_dist.drop('_merge', axis=1, inplace=True)

In [66]:
#suppression colonnes inutiles et renommage 

col_supp_merge3 = ['iso_o', 'dis_int']
isopib_dist.drop(col_supp_merge3, axis=1, inplace=True)

isopib_dist.rename(columns={'iso3': 'ISO', 'pays': 'Pays', "dist":"Distance avec FRA(km)", "city_fr":"Capitale", 'continent': 'Continent', 'area': 'Superficie', 'lat': 'Latitude', 'lon': 'Longitude',}, inplace=True)

In [67]:
#déplacement colonne distance

colonne_distance = isopib_dist.pop('Distance avec FRA(km)')
isopib_dist.insert(isopib_dist.columns.get_loc('Pays') + 1, 'Distance avec FRA(km)', colonne_distance) 

In [68]:
isopib_dist.head(5)

,ISO,Pays,Distance avec FRA(km),Superficie,Continent,Capitale,Latitude,Longitude,PIB (US$)/hab
0,ABW,Aruba,7685.8840,193,America,Oranjestad,12.550000,-70.10000,29007.693003
1,AFG,Afghanistan,5590.3810,652225,Asia,Kaboul,34.516670,69.20000,519.884773
2,AGO,Angola,6510.3220,1246700,Africa,Luanda,-8.833333,13.25000,4095.812942
3,ALB,Albanie,1603.5340,28748,Europe,Tirana,41.333330,19.81667,4531.020806
4,AND,Andorre,711.5485,453,Europe,Andorre-la-Vieille,42.500000,1.50000,38962.880354


In [69]:
# Fonction pour remplacer "E" par "É" en colonne "Pays" tout en maintenant la casse
def replace_e_with_accented_e(text):
    modified_text = ""
    for char in text:
        if char == 'É':
            modified_text += 'E'
        else:
            modified_text += char
    return modified_text

# Liste modifiée
dispalim_pop['Pays'] = dispalim_pop['Pays'].apply(replace_e_with_accented_e)

Fichiers dispalim_pop et isopib_dist

In [70]:
#merge à gauche avec Pays comme PK

main_df =  pd.merge(dispalim_pop, isopib_dist, how='left', on='Pays', indicator=True)

keys_count = main_df['_merge'].value_counts()
print(keys_count)

both          140
left_only      32
right_only      0
Name: _merge, dtype: int64


In [71]:
left_only_rows = main_df[main_df['_merge'] == 'left_only']
left_only_rows

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure(Tonnes),Exportations - Quantité(Tonnes),Importations - Quantité(Tonnes),Nourriture(Tonnes),Production(Tonnes),Population,ISO,Distance avec FRA(km),Superficie,Continent,Capitale,Latitude,Longitude,PIB (US$)/hab,_merge
16,Belgique,44.0,12.65,152000.0,656000.0,338000.0,144000.0,463000.0,1.141975e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18,Bolivie (Etat plurinational de),155.0,36.00,429000.0,1000.0,1000.0,403000.0,533000.0,1.119286e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
24,Bélarus,95.0,27.98,265000.0,152000.0,21000.0,264000.0,463000.0,9.450231e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
26,Cabo Verde,62.0,17.62,10000.0,NaN,12000.0,9000.0,1000.0,5.374980e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,Chine - RAS de Hong-Kong,210.0,53.51,280000.0,663000.0,907000.0,391000.0,24000.0,7.306322e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
32,Chine - RAS de Macao,142.0,34.47,19000.0,0.0,23000.0,21000.0,3000.0,6.225850e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
33,"Chine, Taiwan Province de",126.0,33.17,785000.0,8000.0,161000.0,785000.0,652000.0,2.367455e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
34,"Chine, continentale",59.0,12.33,18161000.0,576000.0,452000.0,17518000.0,18236000.0,1.421022e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
45,El Salvador,71.0,20.43,131000.0,2000.0,16000.0,131000.0,136000.0,6.388126e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
48,Eswatini,22.0,6.46,7000.0,0.0,2000.0,7000.0,6000.0,1.124805e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [72]:
#right_only_rows = main_df[main_df['_merge'] == 'right_only']
#right_only_rows

In [73]:
main_df

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure(Tonnes),Exportations - Quantité(Tonnes),Importations - Quantité(Tonnes),Nourriture(Tonnes),Production(Tonnes),Population,ISO,Distance avec FRA(km),Superficie,Continent,Capitale,Latitude,Longitude,PIB (US$)/hab,_merge
0,Afghanistan,5.0,1.53,57000.0,NaN,29000.0,55000.0,28000.0,36296113.0,AFG,5590.3810,652225.0,Asia,Kaboul,34.516670,69.20000,519.884773,both
1,Afrique du Sud,143.0,35.69,2118000.0,63000.0,514000.0,2035000.0,1667000.0,57009756.0,ZAF,9353.5520,1219912.0,Africa,Le Cap,-33.933330,18.46667,6131.479450,both
2,Albanie,85.0,16.36,47000.0,0.0,38000.0,47000.0,13000.0,2884169.0,ALB,1603.5340,28748.0,Europe,Tirana,41.333330,19.81667,4531.020806,both
3,Algérie,22.0,6.38,277000.0,0.0,2000.0,264000.0,275000.0,41389189.0,DZA,1340.3900,2381741.0,Africa,Alger,36.833330,3.00000,4111.294110,both
4,Allemagne,71.0,19.47,1739000.0,646000.0,842000.0,1609000.0,1514000.0,82658409.0,DEU,439.8984,357325.0,Europe,Essen,51.467000,6.98300,44552.819372,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Emirats arabes unis,147.0,43.47,412000.0,94000.0,433000.0,412000.0,48000.0,9487203.0,ARE,5249.5350,83657.0,Asia,Abou Dhabi,24.466670,54.41667,40644.804043,both
168,Equateur,83.0,19.31,341000.0,0.0,0.0,324000.0,340000.0,16785361.0,ECU,9367.0980,275716.0,America,Quito,-0.233333,-78.50000,6213.501276,both
169,Etats-Unis d'Amérique,219.0,55.68,18266000.0,3692000.0,123000.0,18100000.0,21914000.0,325084756.0,USA,5838.1570,9529106.0,America,Washington,38.867000,-77.00000,60062.222313,both
170,Ethiopie,0.0,0.13,14000.0,NaN,1000.0,14000.0,14000.0,106399924.0,ETH,5582.5020,1127127.0,Africa,Addis Abeba,9.050000,38.83333,768.522308,both


In [74]:
#organisation des colonnes 

col_main_df = ['ISO', 'Pays', 'Continent', 'Capitale', 'Population', 'PIB (US$)/hab', 'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité intérieure(Tonnes)', 'Exportations - Quantité(Tonnes)',
       'Importations - Quantité(Tonnes)', 'Nourriture(Tonnes)',
       'Production(Tonnes)',
       'Distance avec FRA(km)', 'Latitude', 'Longitude'
       ]

main_df = main_df[col_main_df]

In [75]:
main_df

,ISO,Pays,Continent,Capitale,Population,PIB (US$)/hab,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure(Tonnes),Exportations - Quantité(Tonnes),Importations - Quantité(Tonnes),Nourriture(Tonnes),Production(Tonnes),Distance avec FRA(km),Latitude,Longitude
0,AFG,Afghanistan,Asia,Kaboul,36296113.0,519.884773,5.0,1.53,57000.0,NaN,29000.0,55000.0,28000.0,5590.3810,34.516670,69.20000
1,ZAF,Afrique du Sud,Africa,Le Cap,57009756.0,6131.479450,143.0,35.69,2118000.0,63000.0,514000.0,2035000.0,1667000.0,9353.5520,-33.933330,18.46667
2,ALB,Albanie,Europe,Tirana,2884169.0,4531.020806,85.0,16.36,47000.0,0.0,38000.0,47000.0,13000.0,1603.5340,41.333330,19.81667
3,DZA,Algérie,Africa,Alger,41389189.0,4111.294110,22.0,6.38,277000.0,0.0,2000.0,264000.0,275000.0,1340.3900,36.833330,3.00000
4,DEU,Allemagne,Europe,Essen,82658409.0,44552.819372,71.0,19.47,1739000.0,646000.0,842000.0,1609000.0,1514000.0,439.8984,51.467000,6.98300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ARE,Emirats arabes unis,Asia,Abou Dhabi,9487203.0,40644.804043,147.0,43.47,412000.0,94000.0,433000.0,412000.0,48000.0,5249.5350,24.466670,54.41667
168,ECU,Equateur,America,Quito,16785361.0,6213.501276,83.0,19.31,341000.0,0.0,0.0,324000.0,340000.0,9367.0980,-0.233333,-78.50000
169,USA,Etats-Unis d'Amérique,America,Washington,325084756.0,60062.222313,219.0,55.68,18266000.0,3692000.0,123000.0,18100000.0,21914000.0,5838.1570,38.867000,-77.00000
170,ETH,Ethiopie,Africa,Addis Abeba,106399924.0,768.522308,0.0,0.13,14000.0,NaN,1000.0,14000.0,14000.0,5582.5020,9.050000,38.83333


In [76]:
main_df.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\main_df.csv", index=False)

# Remplacement des valeurs manquantes dans main_df

In [77]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 16 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ISO                                                     140 non-null    object 
 1   Pays                                                    172 non-null    object 
 2   Continent                                               140 non-null    object 
 3   Capitale                                                140 non-null    object 
 4   Population                                              172 non-null    float64
 5   PIB (US$)/hab                                           140 non-null    float64
 6   Disponibilité alimentaire (Kcal/personne/jour)          172 non-null    float64
 7   Disponibilité alimentaire en quantité (kg/personne/an)  172 non-null    float64
 8   Disponibilité intérieure(Tonnes)        

In [78]:
#affichage des valeurs des Pays dans les lignes avec des NaN dans la colonne ISO

nan_rows = main_df[main_df['ISO'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes dans la colonne 'ISO'.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'ISO'.")

Il y a 32 valeurs manquantes dans la colonne 'ISO'.
16                                              Belgique
18                       Bolivie (Etat plurinational de)
24                                               Bélarus
26                                            Cabo Verde
31                              Chine - RAS de Hong-Kong
32                                  Chine - RAS de Macao
33                             Chine, Taiwan Province de
34                                   Chine, continentale
45                                           El Salvador
48                                              Eswatini
52                                  Fédération de Russie
68                        Iran (République islamique d')
77                                            Kazakhstan
88                                     Macédoine du Nord
100                                           Monténégro
102                                              Myanmar
108                                 

In [79]:
#remplacement des code ISO3 manquants. Utilisation de Chat GPT qui m'a généré un dictionnaire en associant l'iso du pays selon la liste précédente

iso3_to_country = {
    "Belgique": "BEL",
    "Bolivie (Etat plurinational de)": "BOL",
    "Bélarus": "BLR",
    "Cabo Verde": "CPV",
    "Chine - RAS de Hong-Kong": "HKG",
    "Chine - RAS de Macao": "MAC",
    "Chine, Taiwan Province de": "TWN",
    "Chine, continentale": "CHN",
    "El Salvador": "SLV",
    "Espagne": "ESP",
    "Estonie": "EST",
    "Eswatini": "SWZ",
    "Fédération de Russie": "RUS",
    "Iran (République islamique d')": "IRN",
    "Kazakhstan": "KAZ",
    "Macédoine du Nord": "MKD",
    "Monténégro": "MNE",
    "Myanmar": "MMR",
    "Nouvelle-Calédonie": "NCL",
    "Polynésie française": "PYF",
    "Roumanie": "ROU",
    "Royaume-Uni de Grande-Bretagne et d'Irlande du Nord": "GBR",
    "République centrafricaine": "CAF",
    "République de Corée": "KOR",
    "République de Moldova": "MDA",
    "République dominicaine": "DOM",
    "République démocratique populaire lao": "LAO",
    "République populaire démocratique de Corée": "PRK",
    "République-Unie de Tanzanie": "TZA",
    "Saint-Kitts-et-Nevis": "KNA",
    "Serbie": "SRB",
    "Tchéquie": "CZE",
    "Timor-Leste": "TLS",
    "Venezuela (République bolivarienne du)": "VEN"
}

main_df['ISO'].fillna(main_df['Pays'].map(iso3_to_country), inplace=True)

nan_rows = main_df[main_df['ISO'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'ISO'.")

Il n'y a pas de valeurs manquantes dans la colonne 'ISO'.


In [80]:
#méthodologie identique pour colonne "Continent"

country_to_continent = {
    "Belgique": "Europe",
    "Bolivie (Etat plurinational de)": "America",
    "Bélarus": "Europe",
    "Cabo Verde": "Africa",
    "Chine - RAS de Hong-Kong": "Asia",
    "Chine - RAS de Macao": "Asia",
    "Chine, Taiwan Province de": "Asia",
    "Chine, continentale": "Asia",
    "El Salvador": "America",
    "Espagne": "Europe",
    "Estonie": "Europe",
    "Eswatini": "Africa",
    "Fédération de Russie": "Europe",
    "Iran (République islamique d')": "Asia",
    "Kazakhstan": "Asia",
    "Macédoine du Nord": "Europe",
    "Monténégro": "Europe",
    "Myanmar": "Asia",
    "Nouvelle-Calédonie": "Pacific",
    "Polynésie française": "Pacific",
    "Roumanie": "Europe",
    "Royaume-Uni de Grande-Bretagne et d'Irlande du Nord": "Europe",
    "République centrafricaine": "Africa",
    "République de Corée": "Asia",
    "République de Moldova": "Europe",
    "République dominicaine": "America",
    "République démocratique populaire lao": "Asia",
    "République populaire démocratique de Corée": "Asia",
    "République-Unie de Tanzanie": "Africa",
    "Saint-Kitts-et-Nevis": "America",
    "Serbie": "Europe",
    "Tchéquie": "Europe",
    "Timor-Leste": "Asia",
    "Venezuela (République bolivarienne du)": "America"
}

main_df['Continent'].fillna(main_df['Pays'].map(country_to_continent), inplace=True)

nan_rows = main_df[main_df['Continent'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes dans la colonne 'Continent'.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'Continent'.")

Il n'y a pas de valeurs manquantes dans la colonne 'Continent'.


In [81]:
#idem pour colonne Capitale

pays_capitale = {
    "Belgique": "Bruxelles",
    "Bolivie (Etat plurinational de)": "La Paz",
    "Bélarus": "Minsk",
    "Cabo Verde": "Praia",
    "Chine - RAS de Hong-Kong": "Hong Kong",
    "Chine - RAS de Macao": "Macao",
    "Chine, Taiwan Province de": "Taïpei",
    "Chine, continentale": "Pékin",
    "El Salvador": "San Salvador",
    "Espagne": "Madrid",
    "Estonie": "Tallinn",
    "Eswatini": "Mbabane",
    "Fédération de Russie": "Moscou",
    "Iran (République islamique d')": "Téhéran",
    "Kazakhstan": "Noursoultan",
    "Macédoine du Nord": "Skopje",
    "Monténégro": "Podgorica",
    "Myanmar": "Naypyidaw",
    "Nouvelle-Calédonie": "Nouméa",
    "Polynésie française": "Papeete",
    "Roumanie": "Bucarest",
    "Royaume-Uni de Grande-Bretagne et d'Irlande du Nord": "Londres",
    "République centrafricaine": "Bangui",
    "République de Corée": "Séoul",
    "République de Moldova": "Chișinău",
    "République dominicaine": "Saint-Domingue",
    "République démocratique populaire lao": "Vientiane",
    "République populaire démocratique de Corée": "Pyongyang",
    "République-Unie de Tanzanie": "Dodoma",
    "Saint-Kitts-et-Nevis": "Basseterre",
    "Serbie": "Belgrade",
    "Tchéquie": "Prague",
    "Timor-Leste": "Dili",
    "Venezuela (République bolivarienne du)": "Caracas"
}

main_df['Capitale'].fillna(main_df['Pays'].map(pays_capitale), inplace=True)

nan_rows = main_df[main_df['Capitale'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes dans la colonne 'Capitale'.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'Capitale'.")

Il n'y a pas de valeurs manquantes dans la colonne 'Capitale'.


In [82]:
#idem latitudes

latitudes_cap = {
    "Bruxelles": 50.8503,
    "La Paz": -16.5000,
    "Minsk": 53.9000,
    "Praia": 14.9167,
    "Hong Kong": 22.3000,
    "Macao": 22.2000,
    "Taïpei": 25.0330,
    "Pékin": 35.8617,
    "San Salvador": 13.6989,
    "Madrid": 40.4168,
    "Tallinn": 59.4370,
    "Mbabane": -26.5225,
    "Moscou": 55.7558,
    "Téhéran": 35.6895,
    "Noursoultan": 51.1605,
    "Skopje": 41.9973,
    "Podgorica": 42.4426,
    "Naypyidaw": 21.9162,
    "Nouméa": -22.2675,
    "Papeete": -17.6797,
    "Bucarest": 44.4268,
    "Londres": 51.5074,
    "Bangui": 6.6111,
    "Séoul": 37.5665,
    "Chișinău": 47.0167,
    "Saint-Domingue": 18.4861,
    "Vientiane": 17.9757,
    "Pyongyang": 39.0190,
    "Dodoma": -6.3690,
    "Basseterre": 17.3026,
    "Belgrade": 44.7866,
    "Prague": 50.0755,
    "Dili": -8.5569,
    "Caracas": 10.4910
}

main_df['Latitude'].fillna(main_df['Capitale'].map(latitudes_cap), inplace=True)

nan_rows = main_df[main_df['Latitude'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes dans la colonne 'Latitude'.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'Latitude'.")

Il n'y a pas de valeurs manquantes dans la colonne 'Latitude'.


In [83]:
#idem longitude

longitude_cap = {
    "Bruxelles": 4.3517,
    "La Paz": -68.1193,
    "Minsk": 27.5615,
    "Praia": -23.5087,
    "Hong Kong": 114.1587,
    "Macao": 113.5440,
    "Taïpei": 121.5654,
    "Pékin": 116.4074,
    "San Salvador": -89.1912,
    "Madrid": -3.7038,
    "Tallinn": 24.7536,
    "Mbabane": 31.1300,
    "Moscou": 37.6176,
    "Téhéran": 51.3889,
    "Noursoultan": 71.4128,
    "Skopje": 21.4361,
    "Podgorica": 19.2605,
    "Naypyidaw": 96.2085,
    "Nouméa": 166.4480,
    "Papeete": -149.5690,
    "Bucarest": 26.1025,
    "Londres": -0.1276,
    "Bangui": 18.5820,
    "Séoul": 126.9770,
    "Chișinău": 28.8666,
    "Saint-Domingue": -69.8908,
    "Vientiane": 102.6019,
    "Pyongyang": 125.7490,
    "Dodoma": 35.7382,
    "Basseterre": -62.7196,
    "Belgrade": 20.4489,
    "Prague": 14.4278,
    "Dili": 125.5736,
    "Caracas": -66.8792
}

main_df['Longitude'].fillna(main_df['Capitale'].map(longitude_cap), inplace=True)

nan_rows = main_df[main_df['Longitude'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes dans la colonne 'Longitude'.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'Longitude'.")

Il n'y a pas de valeurs manquantes dans la colonne 'Longitude'.


In [84]:
#remplacement des valeurs manquantes dans la colonne Distance avec FRA(km) en utilisant la formule de distance harvesine 

def haversine(lat1, lon1, lat2, lon2):
    # Rayon de la Terre en kilomètres
    R = 6371.0
    
    # Conversion des coordonnées de degrés en radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Différence de latitude et de longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Formule de la distance haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance en kilomètres
    distance = R * c
    
    return distance

# Coordonnées GPS de Paris
lat1 = 48.8566
lon1 = 2.3522

capitales_coordo = {
    "Bruxelles": (50.8503, 4.3517),
    "La Paz": (-16.5000, -68.1193),
    "Minsk": (53.9000, 27.5615),
    "Praia": (14.9167, -23.5087),
    "Hong Kong": (22.2833, 114.1587),
    "Macao": (22.1987, 113.5440),
    "Taïpei": (25.0330, 121.5654),
    "Pékin": (39.9042, 116.4074),
    "San Salvador": (13.7034, -89.1912),
    "Madrid": (40.4167, -3.7038),
    "Tallinn": (59.4370, 24.7536),
    "Mbabane": (-26.3054, 31.1300),
    "Moscou": (55.7558, 37.6176),
    "Téhéran": (35.6895, 51.3889),
    "Noursoultan": (51.1605, 71.4128),
    "Skopje": (42.0022, 21.4361),
    "Podgorica": (42.4411, 19.2605),
    "Naypyidaw": (19.7633, 96.2085),
    "Nouméa": (-22.2615, 166.4480),
    "Papeete": (-17.6797, -149.5690),
    "Bucarest": (44.4268, 26.1025),
    "Londres": (51.5072, -0.1276),
    "Bangui": (4.3947, 18.5820),
    "Séoul": (37.5665, 126.9770),
    "Chișinău": (47.0269, 28.8666),
    "Saint-Domingue": (18.5392, -69.8908),
    "Vientiane": (17.9667, 102.6019),
    "Pyongyang": (39.0392, 125.7490),
    "Dodoma": (-6.1748, 35.7382),
    "Basseterre": (17.3032, -62.7196),
    "Belgrade": (44.7872, 20.4489),
    "Prague": (50.0755, 14.4278),
    "Dili": (-8.5586, 125.5736),
    "Caracas": (10.4806, -66.8792)
}

distances = []

# Calculer la distance entre Paris et chaque capitale
for capitale, (lat2, lon2) in capitales_coordo.items():
    distance = haversine(lat1, lon1, lat2, lon2)
    distances.append((capitale, distance))

# Convertir la liste de distances en DataFrame
distances_df = pd.DataFrame(distances, columns=["Capitale", "Distance avec FRA(km)"])

In [85]:
# Fusionner les données avec main_df en utilisant la capitale comme PK

distances_df = distances_df.set_index("Capitale")

main_df["Distance avec FRA(km)"] = main_df["Capitale"].map(distances_df["Distance avec FRA(km)"]).fillna(main_df["Distance avec FRA(km)"])

In [86]:
nan_rows = main_df[main_df['Distance avec FRA(km)'].isna()]

if not nan_rows.empty:
    print(f"Il y a {len(nan_rows)} valeurs manquantes dans la colonne 'Distance avec FRA(km)'.")
    print(nan_rows['Pays'])
else:
    print("Il n'y a pas de valeurs manquantes dans la colonne 'Distance avec FRA(km)'.")

Il n'y a pas de valeurs manquantes dans la colonne 'Distance avec FRA(km)'.


In [87]:
#remplacement de toutes les autres valeurs manquantes numériques en 0

main_df.fillna(0, inplace=True)

In [88]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 16 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ISO                                                     172 non-null    object 
 1   Pays                                                    172 non-null    object 
 2   Continent                                               172 non-null    object 
 3   Capitale                                                172 non-null    object 
 4   Population                                              172 non-null    float64
 5   PIB (US$)/hab                                           172 non-null    float64
 6   Disponibilité alimentaire (Kcal/personne/jour)          172 non-null    float64
 7   Disponibilité alimentaire en quantité (kg/personne/an)  172 non-null    float64
 8   Disponibilité intérieure(Tonnes)        

In [89]:
df_final = main_df.reset_index(drop=True)

In [91]:
main_df.to_csv(r"C:\Users\melvin.derouk\Desktop\P9\CSV\main_df", index=False)